# Group Normalization - Text Data

## Giới thiệu

Group Normalization (GN) cho dữ liệu text/sequence thường áp dụng trên sequence data với shape (batch, sequence_length, features).

Group Normalization sẽ normalize theo:
- Chia features thành các groups
- Normalize trong mỗi group (theo features và sequence dimensions trong group đó)
- Không phụ thuộc vào batch size
- Áp dụng cho từng sample độc lập

## Công thức

```
# Chia features thành num_groups groups
group_size = features / num_groups
mean = mean(x, dim=(1, 2))  # Tính mean theo group features và sequence dims
var = var(x, dim=(1, 2))    # Tính variance theo group features và sequence dims
x_norm = (x - mean) / sqrt(var + eps)
output = gamma * x_norm + beta
```

Trong đó:
- num_groups: số lượng groups để chia features
- gamma: learnable scale parameter (một cho mỗi feature)
- beta: learnable shift parameter (một cho mỗi feature)
- eps: small constant để tránh chia cho 0


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class GroupNorm1D:
    """
    Group Normalization cho 1D text/sequence data (3D tensor: batch, sequence_length, features)
    """
    def __init__(self, num_features, num_groups, eps=1e-5):
        self.num_features = num_features
        self.num_groups = num_groups
        self.eps = eps
        
        # Kiểm tra num_features chia hết cho num_groups
        assert num_features % num_groups == 0, f"num_features ({num_features}) phải chia hết cho num_groups ({num_groups})"
        
        self.group_size = num_features // num_groups
        
        # Learnable parameters
        self.gamma = np.ones(num_features)  # Scale
        self.beta = np.zeros(num_features)  # Shift
        
    def forward(self, x):
        """
        x shape: (batch, sequence_length, features)
        """
        batch_size, seq_len, num_features = x.shape
        output = np.zeros_like(x)
        
        # Reshape để group features
        # Shape: (batch, seq_len, num_groups, group_size)
        x_grouped = x.reshape(batch_size, seq_len, self.num_groups, self.group_size)
        
        # Tính mean và var theo group features và sequence dimensions
        # Keep dims: batch, num_groups
        mean = np.mean(x_grouped, axis=(1, 3), keepdims=True)
        var = np.var(x_grouped, axis=(1, 3), keepdims=True)
        
        # Normalize
        x_norm_grouped = (x_grouped - mean) / np.sqrt(var + self.eps)
        
        # Reshape lại về shape ban đầu
        x_norm = x_norm_grouped.reshape(batch_size, seq_len, num_features)
        
        # Scale and shift
        gamma = self.gamma.reshape(1, 1, -1)
        beta = self.beta.reshape(1, 1, -1)
        output = gamma * x_norm + beta
        
        return output

print("GroupNorm1D class đã được định nghĩa!")


## Ví dụ 1: 1 Sample, 1 Feature

Shape: (1, 5, 1) - 1 batch, sequence length 5, 1 feature

Với 1 feature, num_groups phải là 1.


In [ ]:
# Ví dụ 1: 1 Sample, 1 Feature
np.random.seed(42)
x1 = np.random.randn(1, 5, 1) * 10 + 5  # 1 batch, seq_len=5, 1 feature
print(f"Input shape: {x1.shape}")
print(f"Input data:\n{x1[0, :, 0]}")
print(f"\nInput mean: {np.mean(x1):.4f}")
print(f"Input std: {np.std(x1):.4f}")

# Khởi tạo GroupNorm với 1 group
gn1 = GroupNorm1D(num_features=1, num_groups=1)

# Forward pass
output1 = gn1.forward(x1)
print(f"\nOutput shape: {output1.shape}")
print(f"Output data:\n{output1[0, :, 0]}")
print(f"\nOutput mean: {np.mean(output1):.4f}")
print(f"Output std: {np.std(output1):.4f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(x1[0, :, 0], marker='o')
axes[0].set_title('Input (Before GN)')
axes[0].set_xlabel('Sequence Position')
axes[0].set_ylabel('Value')
axes[0].grid(True)

axes[1].plot(output1[0, :, 0], marker='o', color='orange')
axes[1].set_title('Output (After GN)')
axes[1].set_xlabel('Sequence Position')
axes[1].set_ylabel('Value')
axes[1].grid(True)
plt.tight_layout()
plt.show()


## Ví dụ 2: 1 Sample, 2 Features

Shape: (1, 5, 2) - 1 batch, sequence length 5, 2 features

Với 2 features, có thể chia thành 1 hoặc 2 groups.


In [ ]:
# Ví dụ 2: 1 Sample, 2 Features
np.random.seed(42)
x2 = np.random.randn(1, 5, 2) * 10 + 5  # 1 batch, seq_len=5, 2 features
print(f"Input shape: {x2.shape}")
print(f"\nFeature 0 mean: {np.mean(x2[0, :, 0]):.4f}, std: {np.std(x2[0, :, 0]):.4f}")
print(f"Feature 1 mean: {np.mean(x2[0, :, 1]):.4f}, std: {np.std(x2[0, :, 1]):.4f}")

# Khởi tạo GroupNorm với 2 groups (mỗi group 1 feature)
gn2 = GroupNorm1D(num_features=2, num_groups=2)

# Forward pass
output2 = gn2.forward(x2)
print(f"\nOutput shape: {output2.shape}")
print(f"\nFeature 0 - Output mean: {np.mean(output2[0, :, 0]):.4f}, std: {np.std(output2[0, :, 0]):.4f}")
print(f"Feature 1 - Output mean: {np.mean(output2[0, :, 1]):.4f}, std: {np.std(output2[0, :, 1]):.4f}")

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes[0, 0].plot(x2[0, :, 0], marker='o')
axes[0, 0].set_title('Feature 0 Input (Before GN)')
axes[0, 0].set_xlabel('Sequence Position')
axes[0, 0].set_ylabel('Value')
axes[0, 0].grid(True)

axes[0, 1].plot(x2[0, :, 1], marker='o', color='green')
axes[0, 1].set_title('Feature 1 Input (Before GN)')
axes[0, 1].set_xlabel('Sequence Position')
axes[0, 1].set_ylabel('Value')
axes[0, 1].grid(True)

axes[1, 0].plot(output2[0, :, 0], marker='o', color='orange')
axes[1, 0].set_title('Feature 0 Output (After GN)')
axes[1, 0].set_xlabel('Sequence Position')
axes[1, 0].set_ylabel('Value')
axes[1, 0].grid(True)

axes[1, 1].plot(output2[0, :, 1], marker='o', color='red')
axes[1, 1].set_title('Feature 1 Output (After GN)')
axes[1, 1].set_xlabel('Sequence Position')
axes[1, 1].set_ylabel('Value')
axes[1, 1].grid(True)
plt.tight_layout()
plt.show()


## Ví dụ 3: 2 Samples, 1 Feature

Shape: (2, 5, 1) - 2 batches, sequence length 5, 1 feature

Trong trường hợp này, GroupNorm sẽ normalize từng sample độc lập.


In [ ]:
# Ví dụ 3: 2 Samples, 1 Feature
np.random.seed(42)
x3 = np.random.randn(2, 5, 1) * 10 + 5  # 2 batches, seq_len=5, 1 feature
print(f"Input shape: {x3.shape}")
print(f"\nSample 0 mean: {np.mean(x3[0]):.4f}, std: {np.std(x3[0]):.4f}")
print(f"Sample 1 mean: {np.mean(x3[1]):.4f}, std: {np.std(x3[1]):.4f}")

# Khởi tạo GroupNorm với 1 group
gn3 = GroupNorm1D(num_features=1, num_groups=1)

# Forward pass
output3 = gn3.forward(x3)
print(f"\nOutput shape: {output3.shape}")
print(f"\nSample 0 - Output mean: {np.mean(output3[0]):.4f}, std: {np.std(output3[0]):.4f}")
print(f"Sample 1 - Output mean: {np.mean(output3[1]):.4f}, std: {np.std(output3[1]):.4f}")

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes[0, 0].plot(x3[0, :, 0], marker='o')
axes[0, 0].set_title('Sample 0 Input (Before GN)')
axes[0, 0].set_xlabel('Sequence Position')
axes[0, 0].set_ylabel('Value')
axes[0, 0].grid(True)

axes[0, 1].plot(x3[1, :, 0], marker='o', color='green')
axes[0, 1].set_title('Sample 1 Input (Before GN)')
axes[0, 1].set_xlabel('Sequence Position')
axes[0, 1].set_ylabel('Value')
axes[0, 1].grid(True)

axes[1, 0].plot(output3[0, :, 0], marker='o', color='orange')
axes[1, 0].set_title('Sample 0 Output (After GN)')
axes[1, 0].set_xlabel('Sequence Position')
axes[1, 0].set_ylabel('Value')
axes[1, 0].grid(True)

axes[1, 1].plot(output3[1, :, 0], marker='o', color='red')
axes[1, 1].set_title('Sample 1 Output (After GN)')
axes[1, 1].set_xlabel('Sequence Position')
axes[1, 1].set_ylabel('Value')
axes[1, 1].grid(True)
plt.tight_layout()
plt.show()


## Ví dụ 4: 2 Samples, 2 Features

Shape: (2, 5, 2) - 2 batches, sequence length 5, 2 features

Đây là trường hợp đầy đủ nhất với nhiều samples và nhiều features.


In [ ]:
# Ví dụ 4: 2 Samples, 2 Features
np.random.seed(42)
x4 = np.random.randn(2, 5, 2) * 10 + 5  # 2 batches, seq_len=5, 2 features
print(f"Input shape: {x4.shape}")

for b in range(2):
    for f in range(2):
        print(f"Sample {b}, Feature {f} - mean: {np.mean(x4[b, :, f]):.4f}, std: {np.std(x4[b, :, f]):.4f}")

# Khởi tạo GroupNorm với 2 groups (mỗi group 1 feature)
gn4 = GroupNorm1D(num_features=2, num_groups=2)

# Forward pass
output4 = gn4.forward(x4)
print(f"\nOutput shape: {output4.shape}")

for b in range(2):
    for f in range(2):
        print(f"Sample {b}, Feature {f} - Output mean: {np.mean(output4[b, :, f]):.4f}, std: {np.std(output4[b, :, f]):.4f}")

# Visualize
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
for b in range(2):
    for f in range(2):
        axes[b, f*2].plot(x4[b, :, f], marker='o')
        axes[b, f*2].set_title(f'Sample {b}, Feat {f} Input')
        axes[b, f*2].set_xlabel('Seq Pos')
        axes[b, f*2].set_ylabel('Value')
        axes[b, f*2].grid(True)
        
        axes[b, f*2+1].plot(output4[b, :, f], marker='o', color='orange')
        axes[b, f*2+1].set_title(f'Sample {b}, Feat {f} Output')
        axes[b, f*2+1].set_xlabel('Seq Pos')
        axes[b, f*2+1].set_ylabel('Value')
        axes[b, f*2+1].grid(True)

plt.tight_layout()
plt.show()
